In [1]:
import pandas as pd
import gensim

In [2]:
DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'

In [36]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()
len(df)

2386

In [5]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()
tag_list = list(tag_counts.index)

In [ ]:
# tag_documents = tags.str.cat(sep='_').split('_')
# def tokenise(documents):
#     for string in documents:
#         yield gensim.utils.simple_preprocess(string)
# tag_documents = list(tokenise(tag_documents))

# transcript_documents = df['clean_transcript'].map(lambda x: x.tolist())
# transcript_documents = list(transcript_documents)

In [ ]:
# tag_model = gensim.models.Word2Vec(tag_documents,size=150,window=5,min_count=2,workers=4)
# tag_model.train(tag_documents,total_examples=len(tag_documents),epochs=10)

# transcript_model = gensim.models.Word2Vec(transcript_documents,size=150,window=5,min_count=2,workers=4)
# transcript_model.train(transcript_documents,total_examples=len(transcript_documents),epochs=10)

In [25]:
tag_cutoff = int(0.02*len(df.index))

squashed_tags = pd.DataFrame(tag_counts)
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)


In [27]:
squashed_tags

,0
technology,695
science,522
global issues,483
culture,470
design,400
tedx,398
business,329
entertainment,285
health,226
innovation,212


In [53]:
#remove ted tags
ted_tags=[]
for tag in squash_list:
    if 'ted' in tag:
        ted_tags.append(tag)
print(ted_tags)
squashed_tags = squashed_tags.drop(labels = ted_tags)
squash_list = list(squashed_tags.index.values)

['tedx', 'ted fellows', 'ted brain trust']


In [54]:
squashed_tags

,0
technology,695
science,522
global issues,483
culture,470
design,400
business,329
entertainment,285
health,226
innovation,212
education,206


In [29]:
def squashing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [55]:
df['squash_tags'] = df['tags']
df['squash_tags'] = df['squash_tags'].map(lambda x: squashing(x))

In [56]:
df[df['squash_tags']=='']

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags
1307,Jonas Eliasson,How to solve traffic jams,It's an unfortunate reality in nearly every ma...,0:08:27,"cars,transportation",0:12\r\r\rHi. I'm here to talk about congestio...,1525.0,"[hi, talk, congestion, road, congestion, road,...",hi talk congestion road congestion road conges...,
1508,Kelli Swazey,Life that doesn't end with death,"In Tana Toraja, weddings and births aren't the...",0:13:54,"world cultures,death",0:11\r\r\rI think it's safe to say\rthat all h...,1779.0,"[think, safe, human, intimate, death, life, in...",think safe human intimate death life intimacy ...,
1874,Paul Tudor Jones II,Why we need to rethink capitalism,Paul Tudor Jones II loves capitalism. It's a s...,0:09:51,"philanthropy,money,investment",0:12\r\r\rThis is a story about capitalism.\rI...,1442.0,"[story, capitalism, love, success, opportunity...",story capitalism love success opportunity affo...,
1915,Lee Mokobe,A powerful poem about what it feels like to be...,"""I was the mystery of an anatomy, a question a...",0:04:21,"spoken word,poetry,gender,Gender spectrum,Tran...",0:11\r\r\rThe first time I uttered a prayer\rw...,823.0,"[time, utter, prayer, glassstained, cathedral,...",time utter prayer glassstained cathedral kneel...,


In [ ]:
# squash_tags = df['squash_tags'].str.replace(', ', ',').str.lower().str.strip()
# squash_tag_documents = squash_tags.str.cat(sep='_').split('_')
# squash_tag_documents = list(tokenise(squash_tag_documents))

# squash_tag_model = gensim.models.Word2Vec(squash_tag_documents,size=150,window=5,min_count=2,workers=4)
# squash_tag_model.train(squash_tag_documents,total_examples=len(tag_documents),epochs=10)


In [ ]:
# w = 'alternative'
# tag_model.wv.most_similar(positive=w,topn=10)
# # transcript_model.wv.most_similar(positive=w1,topn=10)

In [8]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [63]:
nlp_list = []
for i in range(len(squash_list)):
    nlp_list.append(nlp(squash_list[i]))
# print(nlp_list)

In [62]:
'tedx' in squash_list

False

In [65]:
simlist = []
tagsimcount = [0]*len(squash_list)
tagsimdict = {el:[] for el in squash_list}
for outer in range(len(nlp_list)):
    for inner in range(len(nlp_list)):
        if inner<outer:
            sim = nlp_list[outer].similarity(nlp_list[inner])
            if sim > 0.6:
                ow = squash_list[outer]
                iw = squash_list[inner]
                ow_count = tag_counts.at[ow]
                iw_count = tag_counts.at[iw]
                simlist.append([ow,iw,sim])
                tagsimcount[outer] += 1
                tagsimcount[inner] += 1
                tagsimdict[ow].append(iw)
                tagsimdict[iw].append(ow)
                
items =list(tagsimdict.items())

for k,v in items:
    if v == []:
        del tagsimdict[k]             

# print(simlist)
print(tagsimcount)
print(tagsimdict)
sim_tags = list(tagsimdict.keys())

[3, 8, 7, 1, 1, 1, 0, 6, 4, 7, 1, 5, 9, 0, 5, 5, 2, 8, 1, 0, 2, 3, 3, 1, 1, 0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 6, 0, 3, 0, 0, 0, 0, 1, 1, 7, 4, 4, 1, 4, 0, 0, 1, 3, 0, 1, 0, 1, 1, 0, 0, 4, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 6, 0, 9, 0, 0, 9, 0, 0, 1, 0, 3, 0, 2, 1, 0, 3, 0, 3, 0, 0]
{'technology': ['innovation', 'engineering', 'global development'], 'science': ['education', 'biology', 'economics', 'engineering', 'psychology', 'medical research', 'physics', 'cognitive science'], 'global issues': ['social change', 'future', 'personal growth', 'climate change', 'big problems', 'global development', 'policy'], 'culture': ['society'], 'design': ['architecture'], 'business': ['personal growth'], 'health': ['education', 'medicine', 'health care', 'medical research', 'mental health', 'illness'], 'innovation': ['technology', 'creativity', 'sustainability', 'global development'], 'education': ['science', 'health', 'social change', 'health care', 'medical research', 'mental healt

In [ ]:
# replace tags
# GENERALISE: 'education','teaching' -> 'education'
# REMAIN: 'africa', 'asia' -> 'africa', 'asia'

#anything with ted remove hahaha

In [68]:
#manual removal of tags that shouldn't be replaced
print(sim_tags)

['technology', 'science', 'global issues', 'culture', 'design', 'business', 'health', 'innovation', 'education', 'art', 'society', 'social change', 'politics', 'future', 'creativity', 'biology', 'humanity', 'environment', 'medicine', 'economics', 'brain', 'activism', 'health care', 'music', 'engineering', 'psychology', 'life', 'photography', 'inequality', 'medical research', 'personal growth', 'neuroscience', 'government', 'climate change', 'architecture', 'sustainability', 'disease', 'happiness', 'potential', 'physics', 'evolution', 'mind', 'big problems', 'motivation', 'philosophy', 'biodiversity', 'live music', 'genetics', 'mental health', 'global development', 'cognitive science', 'math', 'religion', 'ecology', 'poverty', 'illness', 'policy']


In [41]:
def sim_counts(a):
#     print(a)
    a_count=tag_counts.at[a]
    all_tags = [a]
    all_count = [a_count]
    b = tagsimdict[a]
    for word in b:
        b_count=tag_counts.at[word]
        all_tags.append(word)
        all_count.append(b_count)
    max_count = max(all_count)
    max_index = all_count.index(max_count)
    major = all_tags[max_index]
    minor = all_tags
    minor.remove(major)
    final = [major]+minor
    return final

In [45]:
## handling 1-to-1
count = 0
one = []
for k,v in tagsimdict.items():
    if len(v) == 1:
        final = sim_counts(k)
        count += 1
#         print(final)
        if final not in one:
            one.append(final)
print(count)
print(len(one))
print(one)

20
16
[['culture', 'society'], ['design', 'architecture'], ['business', 'personal growth'], ['art', 'photography'], ['society', 'humanity'], ['politics', 'activism'], ['music', 'live music'], ['inequality', 'poverty'], ['government', 'policy'], ['disease', 'illness'], ['life', 'happiness'], ['future', 'potential'], ['biology', 'evolution'], ['global issues', 'big problems'], ['creativity', 'motivation'], ['physics', 'math']]


In [72]:
sim_tags_ordered = []
for tag in squash_list:
    if tag in sim_tags:
        sim_tags_ordered.append(tag)
print(sim_tags_ordered)
sim_tags_reversed = sim_tags_ordered
sim_tags_reversed.reverse()
print(sim_tags_reversed)

['technology', 'science', 'global issues', 'culture', 'design', 'business', 'health', 'innovation', 'education', 'art', 'society', 'social change', 'politics', 'future', 'creativity', 'biology', 'humanity', 'environment', 'medicine', 'economics', 'brain', 'activism', 'health care', 'music', 'engineering', 'psychology', 'life', 'photography', 'inequality', 'medical research', 'personal growth', 'neuroscience', 'government', 'climate change', 'architecture', 'sustainability', 'disease', 'happiness', 'potential', 'physics', 'evolution', 'mind', 'big problems', 'motivation', 'philosophy', 'biodiversity', 'live music', 'genetics', 'mental health', 'global development', 'cognitive science', 'math', 'religion', 'ecology', 'poverty', 'illness', 'policy']
['policy', 'illness', 'poverty', 'ecology', 'religion', 'math', 'cognitive science', 'global development', 'mental health', 'genetics', 'live music', 'biodiversity', 'philosophy', 'motivation', 'big problems', 'mind', 'evolution', 'physics', '

In [80]:
final_replacement = {}
for i in range(len(sim_tags_reversed)):
    tag = sim_tags_reversed[i]
    sims = tagsimdict[tag]
    #check if words that main tag is similar too have similar words too
    for sim in sims:
        if sim in final_replacement.keys():
            simlist = final_replacement[sim]
            sims+=simlist
            del final_replacement[sim]
    #remove duplicates
    sims = list(dict.fromkeys(sims))
    final_replacement[tag]=sims
print(final_replacement)

{'government': ['policy', 'global issues', 'social change', 'government'], 'inequality': ['poverty', 'inequality'], 'music': ['live music', 'music'], 'art': ['photography', 'art'], 'health': ['education', 'medicine', 'health care', 'medical research', 'mental health', 'illness', 'science', 'health', 'social change', 'cognitive science', 'biology', 'brain', 'psychology', 'neuroscience', 'physics', 'disease', 'genetics', 'math', 'economics', 'philosophy', 'religion', 'society', 'politics', 'culture', 'humanity', 'global issues', 'future', 'personal growth', 'climate change', 'global development', 'policy', 'activism', 'life', 'potential', 'happiness', 'mind', 'business'], 'business': ['personal growth'], 'design': ['architecture', 'design'], 'culture': ['society'], 'technology': ['innovation', 'engineering', 'global development', 'technology', 'creativity', 'sustainability', 'motivation', 'biodiversity', 'ecology', 'biology', 'global issues', 'social change', 'future', 'environment', 'pe

In [81]:
print(final_replacement.keys())

dict_keys(['government', 'inequality', 'music', 'art', 'health', 'business', 'design', 'culture', 'technology'])


In [101]:
def replacing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in final_replacement.keys():
            final_tags.append(tag)
        else:
            r = []
            for k,v in final_replacement.items():
                if tag in v:
                    r.append(k)
            final_tags += r
    final_tags = list(dict.fromkeys(final_tags))
    final_string = ','.join(final_tags)
    return final_string

In [102]:
df['squash2_tags'] = df['squash_tags']
df['squash2_tags'] = df['squash2_tags'].map(lambda x: replacing(x))


In [107]:
# df.loc[:,['squash_tags','squash2_tags']]
df[df['squash2_tags']=='']

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags,squash2_tags
98,Rives,A mockingbird remix of TED2006,Rives recaps the most memorable moments of TED...,0:04:11,"spoken word,memory,poetry,entertainment,perfor...",0:12\r\r\rMockingbirds are badass.\r\r\r\r\r 0...,782.0,"[mockingbird, badass, mockingbird, mimus, poly...",mockingbird badass mockingbird mimus polyglott...,"entertainment,performance,storytelling",
128,Rives,The 4 a.m. mystery,"Poet Rives does 8 minutes of lyrical origami, ...",0:09:12,"spoken word,poetry,entertainment",0:13\r\r\rThis is a recent comic strip from th...,1405.0,"[recent, comic, strip, los, angeles, time, pun...",recent comic strip los angeles time punch line...,entertainment,
157,Paul Rothemund,Playing with DNA that self-assembles,Paul Rothemund writes code that causes DNA to ...,0:04:59,"MacArthur grant,complexity,computers,DNA",0:12\r\r\rThere's an ancient and universal con...,1012.0,"[ancient, universal, concept, word, power, spe...",ancient universal concept word power spell exi...,computers,
174,Lakshmi Pratury,The lost art of letter-writing,Lakshmi Pratury remembers the lost art of lett...,0:04:09,"relationships,parenting,communication,writing,...","0:11\r\r\rSo I thought, ""I will talk about dea...",601.0,"[think, talk, death, passion, today, actually,...",think talk death passion today actually death ...,"parenting,communication,writing,community",
176,Yossi Vardi,We're worried about local warming ... in your lap,Investor and prankster Yossi Vardi delivers a ...,0:06:15,"comedy,humor",0:12\r\r\rWe are going to talk today about the...,783.0,"[go, talk, today, sequel, inconvenient, truth,...",go talk today sequel inconvenient truth time t...,humor,
195,Jill Sobule + Julia Sweeney,The Jill and Julia Show,"Two TED favorites, Jill Sobule and Julia Sween...",0:06:14,"guitar,singer,comedy,entertainment,storytellin...",0:25\r\r\r Jill Sobule: At a conference in Mon...,991.0,"[jill, sobule, conference, monterey, big, big,...",jill sobule conference monterey big big jellyf...,"entertainment,storytelling,humor,collaboration",
274,Lennart Green,Close-up card magic with a twist,"Like your uncle at a family party, the rumpled...",0:31:08,"magic,illusion,entertainment,humor",0:11\r\r\rMy favorite topic is\rshortcuts.\rTh...,3288.0,"[favorite, topic, shortcut, master, shortcut, ...",favorite topic shortcut master shortcut course...,"entertainment,humor",
289,Carmen Agra Deedy,"Once upon a time, my mother ...","Storyteller Carmen Agra Deedy spins a funny, w...",0:23:34,"memory,children,entertainment,storytelling",0:11\r\r\rWhen I knew I was going to come to s...,3143.0,"[know, go, come, speak, think, get, mother, li...",know go come speak think get mother little cub...,"children,entertainment,storytelling",
449,Felix Dennis,Odes to vice and consequences,"Media big shot Felix Dennis roars his fiery, f...",0:17:24,"poetry,literature,books,history,entertainment,...","0:15\r\r\r""The Better Man.""\r\r\r\r\r 0:23\r\r...",2251.0,"[good, man, good, get, keep, good, spend, spen...",good man good get keep good spend spend good g...,"history,entertainment,writing",
469,Sophal Ear,Escaping the Khmer Rouge,TED Fellow Sophal Ear shares the compelling st...,0:05:57,"Asia,war,history,storytelling,adventure",0:11\r\r\rI normally teach courses on\rhow to ...,907.0,"[normally, teach, course, rebuild, state, war,...",normally teach course rebuild state war today ...,"war,history,storytelling,adventure",


In [109]:
sdf = df[df['squash2_tags']!='']
sdf

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags,squash2_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"culture,politics,science,climate change,enviro...","culture,health,technology,government"
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"invention,engineering,design,global issues","technology,design,government,health"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"poverty,economics,culture,politics,policy,glob...","inequality,health,technology,culture,governmen..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"invention,engineering,entrepreneur,design,busi...","technology,design,business"
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"invention,design,technology,business,art","design,technology,business,art"
5,Craig Venter,Sampling the ocean's DNA,Genomics pioneer Craig Venter takes a break fr...,0:16:51,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several...",2548.0,"[break, ask, people, comment, age, debate, com...",break ask people comment age debate comment un...,"biotech,invention,oceans,genetics,biology,scie...","health,technology"
6,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,0:21:26,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim...",3584.0,"[music, sound, silence, simon, garfunkel, hell...",music sound silence simon garfunkel hello voic...,"computers,music,media,entertainment,performanc...","music,technology"
7,David Rockwell,A memorial at Ground Zero,In this emotionally charged conversation with ...,0:24:37,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,...",4008.0,"[kurt, andersen, like, architect, david, hog, ...",kurt andersen like architect david hog limelig...,"culture,architecture,cities,design,collaboration","culture,design"
8,Dean Kamen,To invent is to give,Inventor Dean Kamen lays out his argument for ...,0:20:07,"robots,cars,industrial design,transportation,i...","0:11\r\r\rAs you pointed out, every time you c...",3278.0,"[point, time, come, learn, morning, world, exp...",point time come learn morning world expert gue...,"robots,invention,education,innovation,social c...","health,technology,government,business"
9,Dean Ornish,The killer American diet that's sweeping the p...,Forget the latest disease in the news: Cardiov...,0:03:18,"obesity,disease,health,health care,culture,foo...",0:11\r\r\rWith all the legitimate concerns\rab...,623.0,"[legitimate, concern, aid, avian, flu, hear, b...",legitimate concern aid avian flu hear brillian...,

In [112]:
s_tags = sdf['squash2_tags'].str.replace(', ', ',').str.lower().str.strip()
# s_split_tags = s_tags.str.split(',')
# tag_counts_per_talk = s_split_tags.apply(len)

s_joined_tags = s_tags.str.cat(sep=',').split(',')
all_stags_w_dup = pd.Series(s_joined_tags)

s_tag_counts = all_stags_w_dup.value_counts()
s_tag_list = list(s_tag_counts.index)

In [113]:
print(len(s_tag_list))
print(s_tag_counts)

9
technology    1871
health        1721
government     658
culture        639
design         414
business       404
art            261
inequality     128
music          126
dtype: int64


In [ ]:
# all_final = []
# for something in all_final_w_dup:
#     if something not in all_final:
#         all_final.append(something)
# print(all_final)

In [ ]:
# for ls in all_final:
#     major = ls[0]
#     print(major)

In [82]:
a = ['x','y','z']
b = ['a','b','c']
a += b
print(a)

['x', 'y', 'z', 'a', 'b', 'c']


{'b': 'y'}


In [46]:
# longtags = []
# for t in squash_list:
#     if ' ' in t:
#         longtags.append(t)
# print(longtags)

In [47]:
# potential = []
# for lt in longtags:
#     split = lt.split()
#     new = []
#     for i in range(len(split)):
#         if split[i] in squash_list:
#             new.append(split[i])
#     if new == []:
#         potential.append(lt)
#     else:
#         potential.append(new)
# print(potential)